In [8]:
import matplotlib.pyplot as plt
import numpy as np
from ase.io import write
from ase.build import nanotube
from ase.neighborlist import NeighborList
from scipy.linalg import eigh
from ase.visualize import view


In [9]:
!pwd

/home/anooja/Dropbox/anooja/EMPA/qtpyt-tests/strained_CNT/cnt_(6,0)


In [10]:
def build_device_structure(n=6, m=0, N_L=3, N_M=4, N_R=3, bond=1.42, direction='x', tag_regions=True):
    """
    Build a CNT device using ASE nanotube with clear region tagging.

    Args:
        n, m: Chiral indices.
        N_L, N_M, N_R: Unit cells in left, middle, right.
        bond: C–C bond length.
        direction: Axis of tube ('x', 'y', or 'z').
        tag_regions: Whether to tag atoms by region (L=1, M=2, R=3).

    Returns:
        ase.Atoms with 'region' tags if enabled.
    """
    total_cells = N_L + N_M + N_R
    atoms = nanotube(n=n, m=m, length=total_cells, bond=bond, verbose=False)


    if direction == 'x':
        atoms.rotate('z', 'x', rotate_cell=True, center='COP')
        axis = 0
    elif direction == 'y':
        atoms.rotate('z', 'y', rotate_cell=True, center='COP')
        axis = 1
    elif direction == 'z':
        axis = 2
    else:
        raise ValueError("Direction must be 'x', 'y', or 'z'")


    positions = atoms.get_positions()
    coord = positions[:, axis]
    x_min, x_max = coord.min(), coord.max()
    total_len = x_max - x_min
    uc_len = total_len / total_cells


    x_L = x_min + N_L * uc_len
    x_M = x_L + N_M * uc_len

    if tag_regions:
        regions = []
        for x in coord:
            if x < x_L:
                regions.append(1)
            elif x < x_M:
                regions.append(2)
            else:
                regions.append(3)
        atoms.set_array('region', np.array(regions, dtype=int))

    atoms.set_pbc((1, 0, 0))
    return atoms

def apply_strain(atoms, strain_percent, axis=0, copy=True):
    """
    Apply uniaxial strain only to region 2 (middle) of the CNT structure,
    anchoring the left boundary and shifting region 3 to preserve bond lengths.

    Parameters
    ----------
    atoms : ase.Atoms
        Atomic structure with a 'region' array.
    strain_percent : float
        Strain percentage to apply to region 2.
    axis : int
        Axis along which to apply strain (default: 0 for x-axis).
    copy : bool
        If True (default), operates on a copy of `atoms`. If False, modifies `atoms` in-place.

    Returns
    -------
    ase.Atoms
        Strained ASE Atoms object.

    Notes
    -----
    Region 2 is stretched from its left boundary.
    Region 3 is rigidly shifted to maintain M–R bond lengths.
    Region 1 is left unmodified.
    """
    if copy:
        atoms = atoms.copy()

    if 'region' not in atoms.arrays:
        raise ValueError("Atoms object must have 'region' array to apply region-specific strain.")

    positions = atoms.get_positions()
    regions = atoms.get_array('region')

    strain = 1 + strain_percent / 100.0
    coord = positions[:, axis]
    mask_M = regions == 2
    mask_R = regions == 3


    x_M_min = coord[mask_M].min()
    x_M_max = coord[mask_M].max()
    original_length = x_M_max - x_M_min


    strained_positions = positions.copy()
    strained_positions[mask_M, axis] = x_M_min + strain * (positions[mask_M, axis] - x_M_min)


    delta = (strain - 1.0) * original_length
    strained_positions[mask_R, axis] += delta

    atoms.set_positions(strained_positions)
    return atoms

def create_cell(atoms, bond=1.42, vacuum=8.0, axis=0, apply_center=True):
    """
    Set the simulation cell for a CNT-like ASE Atoms object with PBC along one axis
    and vacuum in the other directions.

    Parameters
    ----------
    atoms : ase.Atoms
        Atomic structure to update.
    bond : float
        Approximate C–C bond length (used to estimate extra length in periodic direction).
    vacuum : float
        Amount of vacuum (in Å) to place in non-periodic directions.
    axis : int
        Axis (0=x, 1=y, 2=z) along which the CNT lies and PBC is applied.
    apply_center : bool
        Whether to center the atoms inside the new cell.

    Returns
    -------
    ase.Atoms
        The input Atoms object with updated cell.
    """
    if axis not in (0, 1, 2):
        raise ValueError("axis must be 0 (x), 1 (y), or 2 (z)")

    positions = atoms.get_positions()
    coord = positions[:, axis]

    # Determine tube length
    min_pos = coord.min()
    max_pos = coord.max()
    length_axis = max_pos - min_pos + bond * np.cos(np.pi / 3)

    # Build cell with vacuum in other directions
    cell = np.eye(3)
    cell[axis, axis] = length_axis
    cell[(axis + 1) % 3, (axis + 1) % 3] = 2 * vacuum
    cell[(axis + 2) % 3, (axis + 2) % 3] = 2 * vacuum

    atoms.set_cell(cell, scale_atoms=False)
    atoms.set_pbc([axis == i for i in range(3)])

    if apply_center:
        atoms.center()

    return atoms


In [11]:
# Parameters
n, m = 6, 0
ncells = 2
bond = 1.42
onsite = 0.0  # eV
first_neighbor_hopping = -2.7  # eV
beta = 3.37
neighbor_cutoff = 1.43  # Å for nearest neighbors
d0 = bond


In [16]:

N_L = 3
N_M = 28
N_R = 3
device = build_device_structure(n=6, m=0, N_L=N_L, N_M=N_M, N_R=N_R, direction='x')
strain_percent = 4.0
strained_device = apply_strain(device, strain_percent, axis=0)


In [17]:
write(f'structure/cnt_nleads_{N_L}_ncenter_{N_M}_strain_{strain_percent}.xyz', strained_device, format='xyz')

In [7]:
device = create_cell(device, bond=bond, vacuum=8.0, axis=0)
strained_device = create_cell(strained_device, bond=bond, vacuum=8.0, axis=0)

In [ ]:
write("structures/device_init.vasp", device,format='vasp')
write("structures/device_strained.vasp", strained_device,format='vasp')